# XGBoost Evaluation

From our paper on "Explainable Prediction of Acute Myocardial Infarction using Machine Learning and Shapley Values"

In [1]:
# Import libraries 
import pandas as pd
import numpy as np
# !pip install seaborn
import seaborn as sns
import pylab as plt
import pickle
import time
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## Loading Data + XGBoost Model

In [2]:
# Load data
# Import train and test data into dataframes from csv files produced using the data processing code
df_train = pd.read_csv("train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("test.csv", header=None)

# Get data from dataframes
train_y = np.array(df_train[11].values).astype(np.int8)
train_x = np.array(df_train[list(range(11))].values)
test_y = np.array(df_test[11].values).astype(np.int8)
test_x = np.array(df_test[list(range(11))].values)

In [3]:
# load model from file
model = pickle.load(open("xgboost_ecgview.model", "rb"))

## Model Evaluation

In [4]:
start_time = time.time()

## test the model on the test data
pred_test = model.predict(test_x)

# Get f1 score 
f1 = f1_score(test_y, pred_test, average="macro")
print("Test f1 score : %s "% f1)

# Get ROC AUC score 
roc = roc_auc_score(test_y, pred_test)
print("Test ROC AUC score : %s "% roc)

# Get the accuracy
acc = accuracy_score(test_y, pred_test)
print("Test accuracy score : %s "% acc)

print("--- %s seconds ---" % (time.time() - start_time))

Test f1 score : 0.9406646190265121 
Test ROC AUC score : 0.9274943884712142 
Test accuracy score : 0.9489286336449684 
--- 0.19209647178649902 seconds ---


In [5]:
# 10-fold cross validation on the test data 
kfold = StratifiedKFold(n_splits=10, random_state= None)
results = cross_val_score(model, test_x, test_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 93.97% (0.21%)


In [6]:
# Confusion matrix
confusion = confusion_matrix(test_y, pred_test)
sns.heatmap(data=confusion, annot=True, xticklabels=["NotMI", "MI"], yticklabels=["NotMI", "MI"], fmt = "d", annot_kws={"fontsize":16})
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.yticks(va="center")
plt.show()

In [7]:
print(classification_report(test_y, pred_test, digits=4))

              precision    recall  f1-score   support

           0     0.9354    0.9919    0.9628     68865
           1     0.9815    0.8631    0.9185     34461

    accuracy                         0.9489    103326
   macro avg     0.9585    0.9275    0.9407    103326
weighted avg     0.9508    0.9489    0.9480    103326

